In [ ]:
import csv
import pandas as pd
import numpy as np

#The point of this code is to parse through the Bay Area Dataset line by line by converting each row to an ordered dictionary.
#In each iteration, the dictionary is processed to remove unnecessary columns, and if the company name in the dictionary corresponds
#with a company on the warn top 30 companies dataset, I append the dictionary to a final merged dataframe containing warn
#and bay_area columns.

#EDIT: This script has been modified to generate a dataframe with profiles that work in the top 30 commpanies and were not
#laid off

# TODO: Parse top 30 names and convert to list
warn=pd.read_csv(r"C:\Ran\Berkeley\IEOR\290\EmploymentDynamics\ImportantCSVs\top30.csv")
top30_company_names=warn['Company Name'].tolist()
top30_company_tickers = warn['Ticker'].tolist()

warn_matched_companies=[] #intermediate variable used to store rows of the warn dataset that match with the bay area company names
final_merged_dictionary=dict() #Dictionary used to store merged bay_area and warn dictionaries
final_merged_dataframe=pd.DataFrame() #Converting each row of merged dictionary and appending it to this dataframe
final_merged_dictionary2 =dict() #Dictionary used to store merged bay_area and warn dictionaries
final_merged_dataframe2 = pd.DataFrame() #Converting each row of merged dictionary and appending it to this dataframe


with open(r"..\Project\Data\bay_area.csv", encoding='utf-8') as csvfile:
    colnames=['ID', 'Birth Year', 'Gender Flag', 'Skillset1', 'Skillset1 Weight', 'Skillset2', 'Skillset2 Weight', 'City of profile','Country of profile','Education','Elite Institution', 'Start Date', 'StartFlag','End Date', 'EndFlag', 'CurrentEmployFlag','Length','Role','Dept','Company','Company_Norm','Ticker','Exchange','PublicFlag','Location','Industry','EducationFlag','DegreeType','EliteFlag','Dummy1','Dummy2','Dummy3','Dummy4']
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=colnames)
    count=0;

    for idx, row in enumerate(reader):
        if idx % 100000 == 0:
            print(idx)
        #change idx condition to only process the number of rows you want.
        #if idx >= 5000000:
            #break
        #If skillset columns are empty, or if end date is not listed, added
        if row['Skillset1'] == "-1" or row['Skillset2'] == "-1" or row['End Date']=='None':
            continue
        elif row['Company'] in top30_company_names or row['Ticker'] in top30_company_tickers:
            indices = [i for i, (x, y) in enumerate(zip(top30_company_names, top30_company_tickers)) if x == row['Company'] or y == row['Ticker']]  
            
            
            warn_matched_companies=warn.iloc[indices]
            for index,rowwarn in warn_matched_companies.iterrows():
                bay_area_end_date=pd.to_datetime(row['End Date'],infer_datetime_format=True)
                layoff_date=pd.to_datetime(rowwarn['Layoff Date'],infer_datetime_format=True)
                if ((abs(bay_area_end_date-layoff_date) / np.timedelta64(1,'M')) > 6): #Only appending rows 
                    #if layoff and end dates are outside 6 months of each other (leaving not because of layoff)
                    
                    #Converting warn data to a dictionary
                    warn_dict=dict()
                    warn_dict.update({'Layoff Date':rowwarn['Layoff Date']})
                    warn_dict.update({'Company Name':rowwarn['Company Name']})
                    warn_dict.update({'Employees Affected':rowwarn['Employees Affected']})
                    #merging the warn and bay area dictionaries.
                    final_merged_dictionary.update(row)
                    final_merged_dictionary.update(warn_dict)
                    final_merged_dataframe=final_merged_dataframe.append(pd.DataFrame(final_merged_dictionary,columns=final_merged_dictionary.keys(),index=[0]))
                    count=count+1;
                    
                elif ((abs(bay_area_end_date-layoff_date) / np.timedelta64(1,'M')) < 1): #Only appending rows 
                    #if layoff and end dates are within 1 month of each other (leaving because of layoff)
                    
                    #Converting warn data to a dictionary
                    warn_dict2=dict()
                    warn_dict2.update({'Layoff Date':rowwarn['Layoff Date']})
                    warn_dict2.update({'Company Name':rowwarn['Company Name']})
                    warn_dict2.update({'Employees Affected':rowwarn['Employees Affected']})
                    #merging the warn and bay area dictionaries.
                    final_merged_dictionary2.update(row)
                    final_merged_dictionary2.update(warn_dict2)
                    final_merged_dataframe2 = final_merged_dataframe2.append(pd.DataFrame(final_merged_dictionary2,columns=final_merged_dictionary2.keys(),index=[0]))
                    count=count+1;   
            





In [ ]:
final_merged_dataframe['laid_off'] = 0 #classify as "Not laid-off" 
#because the difference between "End- date" and "Layoff- Date" exceed 6 mo
final_merged_dataframe2['laid_off'] = 1 #classify as "laid-off" 
#because the difference between "End- date" and "Layoff- Date" is within 1 mo

In [ ]:
trainingData = pd.concat([final_merged_dataframe, final_merged_dataframe2])
#trainingData.to_csv(...)